In [26]:
# discharge.py usage
# by Rui, Aug 2021

In [1]:
import pickle
from src import discharge
from src import ReadObs
from src import ReadOb
from src import ReadPRD
"""
inputs:
    ----------
    reach:
        - reach['area_fits'] contains the parameters of cross-sectional area change  
        - reach['discharge_models'] contains the parameters of 5 discharge models  
        
    reach_stats: 
        - reach_stats['height'], SWOT reach height [m] 
        - reach_stats['width'], SWOT reach width [m]
        - reach_stats['slope'], SWOT reach slope [m/m]
        
outputs:
    ----------
    d_x_area: cross-sectional area change [m^2]
    d_x_area_u: cross-sectional area change uncertainty [m^2]
    metro_q_c : MetroMan discharge gage constrained [m^3/s]
    metro_q_uc : MetroMan discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    hivdi_q_c : HiVDI discharge gage constrained [m^3/s]
    hivdi_q_uc : HiVDI discharge gage un-constrained [m^3/s]
    momma_q_c : MOMMA discharge gage constrained [m^3/s]
    momma_q_uc : MOMMA discharge gage un-constrained [m^3/s]
    sads_q_c : SADS discharge gage constrained [m^3/s]
    sads_q_uc : SADS discharge gage un-constrained [m^3/s]
    
    **Discharge uncertainties are not implemented at this moment
"""

# load data saved from the RiverObs run on Sac reaches
# note: some of the items under the dict provided in the /data won't be used
obs ='/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/sac_pkl_files/reach25.pkl'
prd ='/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/sac_pkl_files/reach_prd25.pkl'
reach_stats = pickle.load(open(obs, "rb"))
reach = pickle.load(open(prd, "rb"))

In [2]:
# example 1. run discharge module using the .pkl data saved from RiverObs runs
# copied from RiverObs/src/SWOTRiver
discharge_model_values = discharge.compute(
            reach, reach_stats['height'], reach_stats['width'],
            reach_stats['slope'])


print(discharge_model_values)

{'d_x_area': masked_array(data=52.33427927,
             mask=False,
       fill_value=-9999.0), 'd_x_area_u': 11.824505731801548, 'metro_q_uc': -999999999999, 'bam_q_uc': -999999999999, 'hivdi_q_uc': -999999999999, 'momma_q_uc': -999999999999, 'sads_q_uc': -999999999999, 'metro_q_c': masked_array(data=[238.38807826039445],
             mask=[False],
       fill_value=-9999.0), 'bam_q_c': masked_array(data=[213.30025300914113],
             mask=[False],
       fill_value=-9999.0), 'hivdi_q_c': masked_array(data=[238.3870520470281],
             mask=[False],
       fill_value=-9999.0), 'momma_q_c': masked_array(data=[215.14117112199818],
             mask=[False],
       fill_value=-9999.0), 'sads_q_c': masked_array(data=[213.30025300914113],
             mask=[False],
       fill_value=-9999.0)}


/Users/rui/Desktop/Github/offline-discharge-data-product-creation/src/discharge.py:77: RuntimeWarning: invalid value encountered in log10
  log_factor = np.log10((momma_H-momma_B)/(reach_height-momma_B))


In [13]:
# example 2. run discharge module using getting the data saved from SWORD/PRD and rivertile
# read swot rivertile
rivertile_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/rivertile/rivertile.nc'
obs = ReadObs.Rivertile(rivertile_path)

# read prd/sword
sword_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/sword/na_apriori_rivers_v07_4q_v8.nc'
for i in range(len(obs['reach_id'])):
    prd = ReadPRD.ReachDatabase(sword_path, obs['reach_id'][i])
    
    if prd['area_fits']['h_w_nobs'] != -9999:
        discharge_model_values = discharge.compute(
                prd, obs['height'][i], obs['width'][i],
                obs['slope'][i])
        print(i, prd,'obs',obs['height'][i])

14 {'area_fits': {'h_variance': array([-9999.]), 'w_variance': array([-9999.]), 'hw_covariance': array([-9999.]), 'med_flow_area': array([-9999.]), 'h_err_stdev': array([0.1]), 'w_err_stdev': array([5.]), 'h_w_nobs': array([25.]), 'fit_coeffs': array([[[  100.],
        [  100.],
        [  100.]],

       [[-9999.],
        [-9999.],
        [-9999.]]]), 'h_break': array([[-9999.],
       [-9999.],
       [-9999.],
       [-9999.]]), 'w_break': array([[-9999.],
       [-9999.],
       [-9999.],
       [-9999.]])}, 'discharge_models': {'unconstrained': {'MetroMan': {'Abar': array([-9999.]), 'Abar_stdev': array([-9999.]), 'ninf': array([-9999.]), 'ninf_stdev': array([-9999.]), 'p': array([-9999.]), 'p_stdev': array([-9999.]), 'ninf_p_cor': array([-9999.]), 'p_Abar_cor': array([-9999.]), 'ninf_Abar_cor': array([-9999.])}, 'BAM': {'Abar': array([-9999.]), 'n': array([-9999.])}, 'HiVDI': {'Abar': array([-9999.]), 'alpha': array([-9999.]), 'beta': array([-9999.])}, 'MOMMA': {'B': array([-99

22 {'area_fits': {'h_variance': array([1.64844161]), 'w_variance': array([90.0674565]), 'hw_covariance': array([10.46217738]), 'med_flow_area': array([79.32366686]), 'h_err_stdev': array([0.1]), 'w_err_stdev': array([5.]), 'h_w_nobs': array([25.]), 'fit_coeffs': array([[[  3.92500261],
        [ 15.00938491],
        [  5.71553846]],

       [[ 41.81830683],
        [-85.9833434 ],
        [ 25.691015  ]]]), 'h_break': array([[10.35260776],
       [11.52988473],
       [12.01594614],
       [16.96359248]]), 'w_break': array([[ 82.45231929],
       [ 87.07313446],
       [ 94.36861729],
       [122.64708019]])}, 'discharge_models': {'unconstrained': {'MetroMan': {'Abar': array([-9999.]), 'Abar_stdev': array([-9999.]), 'ninf': array([-9999.]), 'ninf_stdev': array([-9999.]), 'p': array([-9999.]), 'p_stdev': array([-9999.]), 'ninf_p_cor': array([-9999.]), 'p_Abar_cor': array([-9999.]), 'ninf_Abar_cor': array([-9999.])}, 'BAM': {'Abar': array([-9999.]), 'n': array([-9999.])}, 'HiVDI': {'Aba

30 {'area_fits': {'h_variance': array([0.13642875]), 'w_variance': array([75.0403992]), 'hw_covariance': array([2.60905896]), 'med_flow_area': array([58.19663494]), 'h_err_stdev': array([0.1]), 'w_err_stdev': array([5.]), 'h_w_nobs': array([25.]), 'fit_coeffs': array([[[   22.70768122],
        [   16.06587318],
        [   53.49312975]],

       [[ -601.8188651 ],
        [ -390.86616948],
        [-1621.79661984]]]), 'h_break': array([[31.76125995],
       [31.7613358 ],
       [32.88861015],
       [33.35418912]]), 'w_break': array([[119.40570103],
       [119.4074234 ],
       [137.51807015],
       [162.42334617]])}, 'discharge_models': {'unconstrained': {'MetroMan': {'Abar': array([-9999.]), 'Abar_stdev': array([-9999.]), 'ninf': array([-9999.]), 'ninf_stdev': array([-9999.]), 'p': array([-9999.]), 'p_stdev': array([-9999.]), 'ninf_p_cor': array([-9999.]), 'p_Abar_cor': array([-9999.]), 'ninf_Abar_cor': array([-9999.])}, 'BAM': {'Abar': array([-9999.]), 'n': array([-9999.])}, 'Hi

In [16]:
# example 3. run discharge module using getting the data saved from SWORD/PRD and single reach
# read swot reach
rivertile_dir = '/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/swot/'
reach_id = 77449100061
ob = ReadOb.Rivertile(rivertile_dir, reach_id)

# read prd/sword
sword_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
creation/data/sword/na_apriori_rivers_v07_4q_v8.nc'

prd = ReadPRD.ReachDatabase(sword_path, ob['reach_id'])
    
for i in range(len(ob['height'])):
    
    if prd['area_fits']['h_w_nobs'] != -9999:
        discharge_model_values = discharge.compute(
                prd, ob['height'][i], ob['width'][i],
                ob['slope'][i])
        print(i, prd,'obs',obs['height'][i])

/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/swot/77449100061_SWOT.nc
0 {'area_fits': {'h_variance': array([2.40597085]), 'w_variance': array([196.87917071]), 'hw_covariance': array([19.8184836]), 'med_flow_area': array([64.54223731]), 'h_err_stdev': array([0.1]), 'w_err_stdev': array([5.]), 'h_w_nobs': array([25.]), 'fit_coeffs': array([[[  2.8255146 ],
        [ 12.13837737],
        [  9.61537714]],

       [[ 51.54043551],
        [-43.20021704],
        [-11.6939711 ]]]), 'h_break': array([[ 7.85858669],
       [10.17309659],
       [12.48761118],
       [14.80211907]]), 'w_break': array([[ 73.74498691],
       [ 80.28466843],
       [108.37911993],
       [130.63398616]])}, 'discharge_models': {'unconstrained': {'MetroMan': {'Abar': array([-9999.]), 'Abar_stdev': array([-9999.]), 'ninf': array([-9999.]), 'ninf_stdev': array([-9999.]), 'p': array([-9999.]), 'p_stdev': array([-9999.]), 'ninf_p_cor': array([-9999.]), 'p_Abar_cor': array([-9999.]), 'ninf_Aba

/Users/rui/Desktop/Github/offline-discharge-data-product-creation/src/discharge.py:77: RuntimeWarning: invalid value encountered in log10
  log_factor = np.log10((momma_H-momma_B)/(reach_height-momma_B))
